In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import calendar

import geopandas
from shapely.geometry import Point

In [2]:
base_dir_path = "Data/Citibike data"
shp_file_path = "Data/zone_grid_data/NYC Taxi Zones/geo_export_517c9e2f-ff9b-4a41-911c-a2d1d835d573.shp"
zones_file_path = "Data/taxi_zone_lookup.csv"

In [3]:
def import_and_combine_citibike_data(base_dir, year):
    year_dir = os.path.join(base_dir, f"{year}-citibike-tripdata")
    all_dataframes = [] 

    # Loop through each month directory in the year directory
    for month in range(1, 13):
        month_dir = os.path.join(year_dir, f"{month}_{calendar.month_name[month]}")
        if os.path.exists(month_dir):  # Check if the month directory exists
            for filename in os.listdir(month_dir):
                if filename.endswith('.csv'):  
                    file_path = os.path.join(month_dir, filename)
                    df = pd.read_csv(file_path)
                    processed_cols = [cols.replace(" ","").lower() for cols in df.columns]
                    df.columns = processed_cols
                    rename_dict = {
                                    'started_at' : 'starttime',
                                    'ended_at' : 'stoptime',
                                    'start_station_name' : 'startstationname',
                                    'start_station_id' : 'startstationid',
                                    'end_station_name' : 'endstationname',
                                    'end_station_id' : 'endstationid',
                                    'start_lat': 'startstationlatitude',
                                    'start_lng': 'startstationlongitude',
                                    'end_lat' : 'endstationlatitude',
                                    'end_lng' : 'endstationlongitude',
                                    'usertype': 'user_member',
                                    'member_casual':'user_member'
                                }
                    existing_rename_dict = {old:new for old,new in rename_dict.items() if old in df.columns}
                    df.rename(columns = existing_rename_dict,inplace=True)
                    drop_columns_overall = ['tripduration','gender','bikeid','birthyear','ride_id','rideable_type']
                    drop_now = [col for col in df.columns if col in drop_columns_overall]
                    df.drop(columns=drop_now,inplace=True)
                    df.dropna(inplace=True)
#                     df['startstationid'] = df['startstationid'].astype(float)
#                     df['endstationid'] = df['endstationid'].astype(float)
                    all_dataframes.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    return combined_df


base_directory = base_dir_path
year_to_import = 2023
combined_data = import_and_combine_citibike_data(base_directory, year_to_import)
# combined_data.head(30)
combined_data.shape

/var/folders/97/nt8v7kl11hgf9sbb8jrhrdlm0000gn/T/ipykernel_30630/1648703790.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/97/nt8v7kl11hgf9sbb8jrhrdlm0000gn/T/ipykernel_30630/1648703790.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/97/nt8v7kl11hgf9sbb8jrhrdlm0000gn/T/ipykernel_30630/1648703790.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/97/nt8v7kl11hgf9sbb8jrhrdlm0000gn/T/ipykernel_30630/1648703790.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/var/folders/97/nt8v7kl11hgf9sbb8jrhrdlm0000gn/T/ipykernel_30630/1648703790.py:12: DtypeWarning: Columns (7) have mixed types. Specify dtype

(36117448, 11)

In [4]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36117448 entries, 0 to 36117447
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   starttime              object 
 1   stoptime               object 
 2   startstationname       object 
 3   startstationid         object 
 4   endstationname         object 
 5   endstationid           object 
 6   startstationlatitude   float64
 7   startstationlongitude  float64
 8   endstationlatitude     float64
 9   endstationlongitude    float64
 10  user_member            object 
dtypes: float64(4), object(7)
memory usage: 3.0+ GB


In [5]:
# temp_data = pd.read_csv("Data/Citib ike Data/2021-citibike-tripdata/1_January/202101-citibike-tripdata_1.csv")
# temp_data.info()

In [6]:
# temp_data2 = pd.read_csv("Data/Citibike Data/2021-citibike-tripdata/12_December/202112-citibike-tripdata_1.csv")
# temp_data2.info()

In [7]:
# temp_data2.isnull().sum()

In [8]:
combined_data

,starttime,stoptime,startstationname,startstationid,endstationname,endstationid,startstationlatitude,startstationlongitude,endstationlatitude,endstationlongitude,user_member
0,2023-01-26 10:53:44,2023-01-26 11:05:17,E 53 St & Madison Ave,6659.03,Columbus Ave & W 72 St,7175.05,40.759724,-73.973664,40.777057,-73.978985,member
1,2023-01-04 11:51:54,2023-01-04 12:07:09,Halsey St & Ralph Ave,4334.02,Clinton Ave & Myrtle Ave,4651.02,40.684970,-73.922755,40.693261,-73.968896,casual
2,2023-01-04 17:26:48,2023-01-04 17:43:29,Bank St & Hudson St,5922.08,W 52 St & 11 Ave,6926.01,40.736566,-74.006092,40.767272,-73.993929,casual
3,2023-01-04 08:31:11,2023-01-04 08:42:12,E 53 St & Madison Ave,6659.03,W 45 St & 8 Ave,6676.02,40.759711,-73.974023,40.759291,-73.988597,member
4,2023-01-03 22:15:34,2023-01-03 22:30:35,Frederick Douglass Blvd & W 112 St,7631.23,Central Park S & 6 Ave,6876.04,40.801694,-73.957145,40.765909,-73.976342,casual
...,...,...,...,...,...,...,...,...,...,...,...
36117443,2023-12-21 18:06:35,2023-12-21 18:17:05,Queens Plaza North & Crescent St,6429.01,3 Ave & E 62 St,6762.04,40.750998,-73.940613,40.763126,-73.965269,member
36117444,2023-12-14 06:42:13,2023-12-14 06:55:47,W 82 St & Central Park West,7304.08,W 52 St & 6 Ave,6740.01,40.782750,-73.971370,40.761330,-73.979820,member
36117445,2023-12-06 08:54:36,2023-12-06 09:16:21,8 Ave & W 33 St,6450.12,3 Ave & E 62 St,6762.04,40.751955,-73.993460,40.763126,-73.965269,casual
36117446,2023-12-22 13:48:37,2023-12-22 13:50:31,9 St & 44 Rd,6361.03,11 St & 44 Rd,6319.04,40.749660,-73.952100,40.749207,-73.949373,member


In [9]:
data = combined_data

In [10]:
def zone_identifier(df, station_id, lat, long):
    localdf = df[[long, lat, station_id]].copy()
    
    shape_df = geopandas.read_file(shp_file_path)
    shape_df.drop(['objectid', 'shape_area', 'shape_leng', 'borough'], axis=1, inplace=True)
    shape_df = shape_df.to_crs({'init': 'epsg:4326'})
    
    local_gdf = geopandas.GeoDataFrame(localdf, crs={'init': 'epsg:4326'},geometry=[Point(xy) for xy in zip(localdf[long], localdf[lat])])

    local_gdf = geopandas.sjoin(local_gdf, shape_df, how='left', op='within')
    
    return local_gdf.zone

In [11]:
def preprocessing(data):
    
    df_preprocess = pd.DataFrame(data=data, columns = data.columns)
#     df_preprocess.dropna(inplace=True)
    df_preprocess.reset_index(inplace=True)
    
    
#     drop_columns_overall = ['tripduration','gender','bikeid','birthyear','ride_id','rideable_type']
#     drop_now = [col for col in df_preprocess.columns if col in drop_columns_overall]
#     df_preprocess.drop(columns=drop_now,inplace=True)
    
    long_lat_id_columns_start_overall = ['start_lng', 'start_lat', 'start_station_id','startstationlongitude', 'startstationlatitude', 'startstationid']
    long_lat_id_columns_end_overall = ['end_lng', 'end_lat', 'end_station_id','endstationlongitude', 'endstationlatitude', 'endstationid']
    
    long_lat_id_columns_start_now = [col for col in df_preprocess.columns if col in long_lat_id_columns_start_overall]
    long_lat_id_columns_end_now = [col for col in df_preprocess.columns if col in long_lat_id_columns_end_overall]
    
    df_preprocess['start_zone'] = zone_identifier(df_preprocess, long_lat_id_columns_start_now[0],long_lat_id_columns_start_now[1],long_lat_id_columns_start_now[2])
    df_preprocess['end_zone'] = zone_identifier(df_preprocess, long_lat_id_columns_end_now[0],long_lat_id_columns_end_now[1],long_lat_id_columns_end_now[2])
    
#     print(df_preprocess['start_zone'])
#     print(df_preprocess['end_zone'])
    
    zone_file = pd.read_csv(zones_file_path)
    zone_df = pd.DataFrame(zone_file,columns=["Zone"])
    zone_map_inverse = {v:k+1 for k,v in enumerate(zone_df['Zone'])}
    zone_map = {k+1:v for k,v in enumerate(zone_df['Zone'])}
    
    df_preprocess['start_zone_encoded'] = [zone_map_inverse[v] for v in df_preprocess["start_zone"]]
    df_preprocess['end_zone_encoded'] = [zone_map_inverse[v] for v in df_preprocess["end_zone"]]
    
#     print(df_preprocess['start_zone_encoded'])
#     print(df_preprocess['end_zone_encoded'])
    
    df_preprocess.drop(columns = ['start_zone','end_zone'],inplace=True)
    
    df_preprocess.drop(columns = long_lat_id_columns_start_now + long_lat_id_columns_end_now, inplace=True)
    
    start_time_cols_overall = ['starttime','started_at']
    stop_time_cols_overall = ['stoptime','ended_at']
    
    start_time_now = [col for col in df_preprocess.columns if col in start_time_cols_overall]
    stop_time_now = [col for col in df_preprocess.columns if col in stop_time_cols_overall]
    
    df_preprocess[start_time_now[0]] = pd.to_datetime(df_preprocess[start_time_now[0]])
    df_preprocess['unix_start_time'] = df_preprocess[start_time_now[0]].astype('int64') // 10**9

    df_preprocess[stop_time_now[0]] = pd.to_datetime(df_preprocess[stop_time_now[0]])
    df_preprocess['unix_stop_time'] = df_preprocess[stop_time_now[0]].astype('int64') // 10**9
    
    df_preprocess.drop(columns = start_time_now + stop_time_now, inplace=True)
    
    df_preprocess['diff'] = df_preprocess['unix_stop_time'] - df_preprocess['unix_start_time']
    df_preprocess = df_preprocess[df_preprocess['diff'] <= 3600]
    
    user_member_type = {
        'Subscriber': 0,
        'member' : 0,
        'Customer' : 1,
        'casual' : 1
    }
    
#     user_member_var = ["usertype","member_casual"]
#     user_member_current_var = [col for col in df_preprocess.columns if col in user_member_var]
    
#     temp2 = [value for col in user_member_current_var for value in df_preprocess[col]]
    
#     df_preprocess['user_member_encoded'] = [user_member_type[string] for string in df_preprocess[user_member_current_var[0]]]
    df_preprocess['user_member_encoded'] = [user_member_type[string] for string in df_preprocess["user_member"]]

    df_preprocess = df_preprocess[df_preprocess['start_zone_encoded']!=df_preprocess['end_zone_encoded']]
    df_processed = df_preprocess.sort_values(by='unix_start_time')
    df_processed.reset_index(inplace=True)
    final_processed = df_processed[['start_zone_encoded','end_zone_encoded','unix_start_time','unix_stop_time','user_member_encoded']]
    for col in final_processed.columns:
        final_processed[col] = final_processed[col].astype(int)
    return final_processed

In [ ]:
export_df = preprocessing(data)
export_df

/Users/neelagarwal/anaconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
# export_df.isnull().sum()
# export_df.to_csv("ProcessedCitibikeData/CitibikeProcessed2023.csv",index=False)